In [19]:
from PIL import Image
import urllib.request
import io
from skimage import io
from ipywidgets import Image
import pandas as pd
import time 
import ast
from all_functions import *
import requests
from pylatex import Document, Section, Subsection, Command
from pylatex.utils import italic, NoEscape
from selenium import webdriver

In [33]:
scrap_params = {
    'results_per_site' : 10,
    'job_title' : 'data+science',
    'city' : 'Boston',
    'site_list' : ['stack', 'indeed', 'linkedin'],
    'linked_in_username' : 'YOUR_USERNAME',
    'linked_in_password' : 'YOUR_USERNAME',
    'chromedriver_location' : u'YOURCHROMEDRIVERLOC',
    'api_key' : 'YOURAPIKEY'
}
APIkey = scrap_params['api_key']

In [34]:
def get_address(query):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        key = APIkey
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + key
        my_dict = requests.get(url_search).json()
        return my_dict

In [35]:
def get_commute(dest, origin):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=' + APIkey
    
    url_search = base + origin + dest_str + api_key
    my_dict = requests.get(url_search).json()
    return my_dict

In [36]:
def dist_duration(company_list, id_list):
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    for index, company in enumerate(company_list):
        comm_dict = {}
        company = company
        tmp_id = id_list[int(index)]
        text_dict = get_address(str(company))
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            continue
        else:
            address = text_dict['candidates'][0]['formatted_address']
            place = text_dict['candidates'][0]['name']    
            res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])

        comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street')
        if comm['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_commute.')
            continue
        else:
            try:
                distance = (comm['rows'][0]['elements'][0]['distance']['text'])
                duration = (comm['rows'][0]['elements'][0]['duration']['text'])
            except:
                #print(comm)
                break

        comm_dict['company'] = company
        comm_dict['place'] = place
        comm_dict['address'] = address
        comm_dict['distance'] = distance
        comm_dict['duration'] = duration
        comm_dict['sID'] = tmp_id
        comm_tmp_df = pd.DataFrame(comm_dict, index = [comm_dict['sID']])
        comm_df = comm_df.append(comm_tmp_df, sort=False)
    return comm_df

In [38]:
def ult_scrap(**scrap_params):
    results = scrap_params['results_per_site']
    job_title = scrap_params['job_title']
    city = scrap_params['city'] 
    site_list = scrap_params['site_list']
    linked_in_username = scrap_params['linked_in_username'] 
    linked_in_password = scrap_params['linked_in_password']
    chromedriver_location = scrap_params['chromedriver_location'] 
     #api_key = scrap_params[''] 
    
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    job_col_list = ['site', 'sID',  'title', 'company', 'location', 'salary', 'summary', 'tags','id', 'comp_loc', 'link', 'pulldate','postdate', 'ezapply']
    job_df = pd.DataFrame(columns=job_col_list).set_index('sID')
    #tmp_df = pd.DataFrame(columns=job_col_list, index=range(0,results))
    
    #if(file > SKIPPER):
        # for results
    num = 0
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-extensions")
    options.add_argument('--log-level=3')
    #wd = webdriver.Chrome(u'/Users/matt/Documents/STUFF/chromedriver/chromedriver.exe', options=options)
    wd = webdriver.Chrome(chromedriver_location, options=options)
    for site in site_list: 
        site_ind = results
        
        if site == 'stack':
            site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
            list_o_divs = [] # holds divs if multiple pages
            tmp_dict = {} # holds info on each div loop
            # https://stackoverflow.com/jobs?q=data+scientist&l=boston&d=20&u=Miles
            pg = 1 # page counter 
            base = 'https://stackoverflow.com/jobs?'
            tot_pg = ((results // 20) + 1)
            print('Stack overflow total pages: ' + str(tot_pg))
            while pg <= tot_pg: 
                params = {'q' : job_title, 'sort' : 'p', 'l' : city, 'd' : '20', 'u': 'Miles', 'pg': pg} 
                page = requests.get(base, params) 
                time.sleep(1)  
                soup = get_soup(page.text)
                divs = soup.find_all(name="div", attrs={"class":"-job-summary"})
                list_o_divs.append(divs)
                pg = pg + 1
            for divs in list_o_divs:
                for div in divs:
                    if (site_ind < results): # if there is a div past the number of results for the site, doesn't add it to tmp_dict
                        tmp_dict['sID'] = 'ID' + str(num)
                        tmp_dict['site'] = site
                        tmp_dict['pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                        tmp_dict['link'] = extract_link(div, site)
                        tmp_dict['title'] = extract_job_title(div, site)
                        tmp_dict['summary'] = extract_summary(div, site)
                        tmp_dict['salary'] = extract_salary(div, site)
                        tmp_dict['location'] = extract_location(div, site).strip()
                        tmp_dict['tags'] = extract_tags(div, site)
                        tmp_dict['id'] = extract_id(div, site)
                        tmp_dict['ezapply'] = extract_easyapply(div, site)
                        tmp_dict['postdate'] = extract_postdate(div, site)
                        tmp_dict['company'] = extract_company(div, site).split('-')[0].strip()
                        tmp_dict['comp_loc'] = extract_company(div, site)
                        #tmp_df.set_index('sID', inplace = True)
                        tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                        job_df = job_df.append(tmp_df, sort=False)
                        num = num + 1
                        site_ind = site_ind + 1
            
        if site == 'indeed':
            indeed_pg_num = 0
            site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
            list_o_divs = [] # holds divs if multiple pages
            tmp_dict = {} # holds info on each div loop
            # https://www.indeed.com/jobs?q=data+science&l=boston&sort=date
            res_left = results
            base = 'https://www.indeed.com/jobs?'
            while res_left > 0:            
                params = {'q' : job_title,'l' : city, 'start' : (results-res_left),  'sort' : 'date', 'radius': '25'}
                page = requests.get(base, params) 
                time.sleep(1)  
                soup = get_soup(page.text)
                divs = soup.find_all(name="div", attrs={"class":"row"})
                list_o_divs.append(divs)
                res_left = res_left - len(divs)
                indeed_pg_num = indeed_pg_num + 1
            print('total indeed pages : ' + str(indeed_pg_num))
            for divs in list_o_divs:
                for div in divs:
                    if (site_ind < results):
                        tmp_dict['sID'] = 'ID' + str(num)
                        tmp_dict['site'] = site
                        tmp_dict['pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                        tmp_dict['link'] = extract_link(div, site)
                        tmp_dict['title'] = extract_job_title(div, site)
                        tmp_dict['summary'] = extract_summary(div, site)
                        tmp_dict['salary'] = extract_salary(div, site)
                        tmp_dict['location'] = extract_location(div, site).strip()
                        tmp_dict['tags'] = extract_tags(div, site)
                        tmp_dict['id'] = extract_id(div, site)
                        tmp_dict['ezapply'] = extract_easyapply(div, site)
                        tmp_dict['postdate'] = extract_postdate(div, site)
                        tmp_dict['company'] = extract_company(div, site).strip()
                        tmp_dict['comp_loc'] = str(extract_company(div, site).strip() + ', ' + extract_location(div, site).strip())
                        tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                        job_df = job_df.append(tmp_df, sort=False)
                        num = num + 1
                        site_ind = site_ind + 1
                        
        if site == 'linkedin':
            list_o_divs = []
            tot_pg = ((results // 25) + 1)
            res_left = results
            print('linkedin total pages: ' + str(tot_pg))
            # LOGIN 
            home_url = 'https://www.linkedin.com/'
            wd.get(home_url)
            login_email = wd.find_elements_by_class_name('login-email')[0].send_keys(linked_in_username)
            login_password = wd.find_elements_by_class_name('login-password')[0].send_keys(linked_in_password)
            wd.find_elements_by_class_name('submit-button')[0].click
            
            # Get info
            for i in range(tot_pg):
                url = 'http://www.linkedin.com/jobs/search/?keywords=data+science&location=Boston&sortBy=DD&start=' + str(i*25)
                wd.get(url)
                listings = wd.find_elements_by_xpath("//li[@class='job-listing']")
                res_len = min(len(listings), int(res_left))

                links = [l.get_attribute('href') for l in wd.find_elements_by_xpath("//a[@class='job-title-link']")]
                titles = [t.text for t in wd.find_elements_by_xpath("//span[@class='job-title-text']")]
                locations = [L.text for L in wd.find_elements_by_xpath("//span[@class='job-location']/span")]
                companies = [c.text for c in wd.find_elements_by_xpath("//span[@class='company-name-text']")]
                descr = [d.text for d in wd.find_elements_by_xpath("//div[@class='job-description']")]
                ezapply = [e.text for e in wd.find_elements_by_xpath("//div[@class='job-flavor-in-apply-container']")]
                comp_loc = []
                if res_len < 25:
                    links = links[:res_len]
                    titles = titles[:res_len]
                    locations = locations[:res_len]
                    companies = companies[:res_len]
                    descr = descr[:res_len]
                    ezapply = ezapply[:res_len]
                sID = [('ID' + str(n)) for n in range(num, num + res_len)]
                site = ['linkedin' for n in range(num, num + res_len)]
                pulldates = [str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time()))) for n in range(num, num + res_len)]
                for i, elem in enumerate(companies):
                    comp_loc.append(elem + ', ' + locations[i])
                tmp_df = pd.DataFrame({'sID': sID, 'site' : site, 'title' : titles, 'company' : companies, 
                                       'location' : locations,  'link' : links, 'ezapply' : ezapply, 'summary' : descr,
                                      'pulldate' : pulldates, 'comp_loc': comp_loc}).set_index('sID')

                num += len(tmp_df)
                res_left = res_left - len(tmp_df)
                job_df = job_df.append(tmp_df, sort=False)
            
    companies = job_df['comp_loc']
    ids = job_df.index.tolist()
    comm_df = dist_duration(companies, ids)

    fin_df = job_df.merge(comm_df, how = 'left', left_index=True, right_index=True)
    fin_df.rename({'company_x' : 'company'}, axis='columns', inplace=True)
    job_col_list = ['site', 'sID',  'title', 'company', 'location', 'salary', 'summary', 'tags','id', 'comp_loc', 'link', 'pulldate','postdate', 'ezapply']
    fin_df = fin_df[['title', 'comp_loc', 'distance', 'duration', 'summary', 'tags', 'link', 'pulldate', 'company', 'address', 'site']]
    file = file + 1
    wd.close()
    return fin_df

In [39]:
fetch = ult_scrap(**scrap_params)
fetch

Stack overflow total pages: 1
total indeed pages : 1
linkedin total pages: 1
Armored Things             - Boston, MA         has 0 results in get_address.
Armored Things             - Boston, MA         has 0 results in get_address.


,title,comp_loc,distance,duration,summary,tags,link,pulldate,company,address,site
ID0,Big Data Hadoop Engineer,Eagle Investment Systems - Wellesl...,7.2 mi,21 mins,No summary on stack,"hadoop, apache-spark, python, hbase, apache-ka...",https://www.stackoverflow.com/jobs/198218/big-...,11-01-2018_03-36_PM,Eagle Investment Systems,"45 William St, Wellesley, MA 02481, USA",stack
ID1,Senior Software Engineer,"Klaviyo - Boston, MA",5.2 mi,17 mins,No summary on stack,"python, django, amazon-web-services, mysql, ca...",https://www.stackoverflow.com/jobs/143790/seni...,11-01-2018_03-36_PM,Klaviyo,"225 Franklin St floor 10, Boston, MA 02110, USA",stack
ID2,Mobile Developer for iOS and Android,"Armored Things - Boston, MA",NaN,NaN,No summary on stack,"ios, android, react-native, swift, kotlin,",https://www.stackoverflow.com/jobs/203833/mobi...,11-01-2018_03-36_PM,Armored Things,NaN,stack
ID3,Front-End/Full Stack Developer,Eagle Investment Systems - Wellesl...,7.2 mi,21 mins,No summary on stack,"saas, cloud, javascript, java, python,",https://www.stackoverflow.com/jobs/198217/fron...,11-01-2018_03-36_PM,Eagle Investment Systems,"45 William St, Wellesley, MA 02481, USA",stack
ID4,Director of Data Science,"Wellframe - Boston, MA",4.2 mi,17 mins,No summary on stack,"machine-learning, r, analytics, bigdata, stati...",https://www.stackoverflow.com/jobs/206439/dire...,11-01-2018_03-36_PM,Wellframe,"321 Summer St 7th floor, Boston, MA 02210, USA",stack
ID5,Backend Software Engineer,"TrueMotion - Boston, MA",4.0 mi,15 mins,No summary on stack,"amazon-web-services, scrum, python, rest, post...",https://www.stackoverflow.com/jobs/206278/back...,11-01-2018_03-36_PM,TrueMotion,"470 Atlantic Ave, Boston, MA 02210, USA",stack
ID6,Lead Backend Software Engineer,"TrueMotion - Boston, MA",4.0 mi,15 mins,No summary on stack,"amazon-web-services, scrum, python, postgresql,",https://www.stackoverflow.com/jobs/206188/lead...,11-01-2018_03-36_PM,TrueMotion,"470 Atlantic Ave, Boston, MA 02210, USA",stack
ID7,Senior Front-end Engineer,MGH & BWH Center for Clinical Data Science ...,1.8 mi,10 mins,No summary on stack,"reactjs, javascript, css, html5,",https://www.stackoverflow.com/jobs/172895/seni...,11-01-2018_03-36_PM,MGH & BWH Center for Clinical Data Science,"388 Commonwealth Avenue, Boston, MA 02215, USA",stack
ID8,Software Engineer - Physiological Sensing and ...,Charles River Analytics Inc - Camb...,6.0 mi,18 mins,No summary on stack,"java, python, node.js, html5, css3,",https://www.stackoverflow.com/jobs/196479/soft...,11-01-2018_03-36_PM,Charles River Analytics Inc,"625 Mt Auburn St # 3, Cambridge, MA 02138, USA",stack
ID9,Devops Engineer,"Armored Things - Boston, MA",NaN,NaN,No summary on stack,"python, amazon-web-services, kubernetes, azure...",https://www.stackoverflow.com/jobs/203849/devo...,11-01-2018_03-36_PM,Armored Things,NaN,stack


NameError: name 'wd' is not defined